Jadi latar belakang permasalahanya adalah DQLab Telco merupakan perusahaan Telco yang sudah mempunyai banyak cabang tersebar dimana-mana. 
DQLab Telco konsisten untuk memperhatikan customer experience-nya sehingga tidak akan ditinggalkan pelanggan. Jadi ada cukup banyak pelanggan yang hjijrah ke kompetitornya.
Pihak manajemen tu pengen ngurangi jumlah pelanggan yang beralih atau biasa kita sebut *churn* dg menggunakan ml.
Makanya tim ds disuruh utk mpersiapkan data lalu buat model prediksi utk nentuin pelanggan itu bakalan churn/ga

kita disuruh nyiapin data buat sblm melakukan modeling
Langkah yang akan dilakukan adalah,

- Mencari ID pelanggan (Nomor telepon) yang valid
- Mengatasi data-data yang masih kosong (Missing Values)
- Mengatasi Nilai-Nilai Pencilan (Outlier) dari setiap Variable
- Menstandardisasi Nilai dari Variable

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

 detail datanya adalah sebagai berikut:

- UpdatedAt Periode of Data taken
- customerID Customer ID
- gender Whether the customer is a male or a female (Male, Female)
- SeniorCitizen Whether the customer is a senior citizen or not (1, 0)
- Partner Whether the customer has a partner or not (Yes, No)
- Dependents Whether the customer has dependents or not (Yes, No)
- tenure Number of months the customer has stayed with the company
- PhoneService Whether the customer has a phone service or not (Yes, No)
- MultipleLines Whether the customer has multiple lines or not (Yes, No, No phone service)
- InternetService Customer’s internet service provider (DSL, Fiber optic, No)
- OnlineSecurity Whether the customer has online security or not (Yes, No, No internet service)
- OnlineBackup Whether the customer has online backup or not (Yes, No, No internet service)
- DeviceProtection Whether the customer has device protection or not (Yes, No, No internet service)
- TechSupport Whether the customer has tech support or not (Yes, No, No internet service)
- StreamingTV Whether the customer has streaming TV or not (Yes, No, No internet service)
- StreamingMovies Whether the customer has streaming movies or not (Yes, No, No internet service)
- Contract The contract term of the customer (Month-to-month, One year, Two year)
- PaperlessBilling Whether the customer has paperless billing or not (Yes, No)
- PaymentMethod The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- MonthlyCharges The amount charged to the customer monthly
- TotalCharges The total amount charged to the customer
- Churn Whether the customer churned or not (Yes or No)

In [4]:
# importing data

pd.options.display.max_columns = 50
df = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqlab_telco.csv')
print(f'data memiliki ukuran sejumlah {df.shape[0]} baris dan {df.shape[0]} kolom')

data memiliki ukuran sejumlah 7113 baris dan 7113 kolom


In [5]:
df.head()

,UpdatedAt,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,202006,45759018157,Female,0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,202006,45557574145,Male,0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,202006,45366876421,Male,0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,202006,45779536532,Male,0,No,No,45.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,202006,45923787906,Female,0,No,No,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.customerID.nunique() # mencari data customer ID yang bersifat unique

7017

In [10]:
df['valid_id'] = df['customerID'].astype(str).str.match(r'(45\d{9,10})')

In [11]:
df = (df[df['valid_id'] == True]).drop('valid_id', axis = 1)

In [12]:
df['customerID'].count() # setelah difilter

7006

In [14]:
'''
biasa itu ada 2 tipe yaitu 
- Duplikasi dikarenakan inserting melebihi satu kali dengan nilai yang sama tiap kolomnya
- Duplikasi dikarenakan inserting beda periode pengambilan data
'''


df.drop_duplicates()
# Drop duplicate ID sorted by Periode
df = df.sort_values('UpdatedAt', ascending=False).drop_duplicates('customerID') #sort value buat ngecek pengambilan dt terakhir
df['customerID'].count() # hasil jumlah customer ID setelah dihilangkan dupliksi(distinct)

6993

Validitas dari ID Number pelanggan sangat diperlukan untuk memastikan bahwa data yang kita ambil sudah benar. Berdasarkan hasil tersebut, terdapat perbedaan jumlah nomor ID dari data pertama kali di load sampai dengan hasil akhir. Jumlah row data ketika pertama kali di load ada sebanyak 7113 rows dan 22 columns dengan 7017 jumlah ID yang unique. Kemudian setelah di cek validitas dari ID pelanggan, maka tersisa 6993 rows data